# Groundwater Modelling in Python - Session 2

<img src="figs/part_of_cover_bakker_post.png" width="800px"> 

In [ ]:
# Before starting we need to import NumPy and Matplotlib and set some defaults
import numpy as np
import matplotlib.pyplot as plt
# and set some parameters to make the figures look good
plt.rcParams["figure.autolayout"] = True # same as tight_layout after every plot
plt.rcParams["figure.figsize"] = (9, 3.5) # set default figure size
plt.rcParams["contour.negative_linestyle"] = 'solid' # set default line style
plt.rc('font', size=12)

Sections and Exercises with a star are additional material that is not covered in detail during the workshop.

# Introduction

This session deals with one-dimensional transient flow, which means that the head and flow are a function of $x$ and time $t$, respectively. As for steady flow, the starting point of the derivation of the differential equation is the volume balance for a small piece of the aquifer
\begin{equation}\label{continuity}
\text{Volume in} - \text{Volume out} = \text{Increase in volume}
\end{equation}
Recall from the previous session that the inflow consists of horizontal flow from the left, $Q_x(x, t)$ [L$^2$/T], and recharge at the top, $N$ [L/T] and that outflow consists of horizontal flow at the right, $Q_x(x+\Delta x, t)$ (see Figure). 

<IMG src="figs/merged_nbs5_4_0.png"  width=400>>

Unlike for steady flow, the change in storage within the volume is no longer zero. The increase in storage equals the increase in head multiplied by the storage coefficient $S$, so
\begin{equation}
Q_x(x,t)\Delta t + N\Delta x \Delta t - Q_x(x+\Delta x,t)\Delta t =
S\Delta x[h(x, t + \Delta t) - h(x, t)]
\end{equation}
Division by $\Delta x \Delta t$ and rearrangement of terms gives
\begin{equation}
\frac{Q_x(x + \Delta x, t) - Q_x(x, t)}{\Delta x}
= -S \frac{h(x, t + \Delta t) - h(x, t)}{\Delta t} + N
\end{equation}
In the limit for $\Delta x \to 0$ and $\Delta t \to 0$ the volume balance turns into the differential equation for one-dimensional horizontal transient flow
\begin{equation}\label{basicdeq1dtr}
\frac{\partial Q_x}{\partial x} = - S\frac{\partial h}{\partial t} + N
\end{equation}

Substitution of $Q_x = -T\tfrac{\partial h}{\partial x}$ gives
\begin{equation} \label{deqtransient}
\frac{\partial^2 h}{\partial x^2} = \frac{S}{T} \frac{\partial h}{\partial t} - \frac{N}{T}
\end{equation}
The term $T/S$ is also referred to as the aquifer diffusivity.

# Solution 1. Riverbank storage

## Problem definition
The figure below shows an aquifer bounded on the left by a fully penetrating canal or river that is in direct hydraulic contact with the aquifer. Towards the right, the aquifer has an infinite extent in the $x$ direction. Initially, the head in the aquifer is equal to the canal level $h_0$ everywhere. At time $t=t_0$, the stage in the canal is raised by an amount $\Delta h$. It is further assumed that the head change does not extend indefintively inland, so $h|_{x\to\infty, t\ge 0} = h_0$.

<img src="figs/fig5.1.png" width=400>

The solution for the head under these conditions is
\begin{equation}\label{hedelman}
h(x, t) = \Delta h \, \text{erfc}(u) + h_0 {\hskip 2em} t>t_0
\end{equation}
where 
\begin{equation}\label{uedelman}
u = \sqrt{\frac{Sx^2}{4T(t-t_0)}}
\end{equation}
and erfc is the complimentary error function
\begin{equation}
\text{erfc}(u) = \int_u^\infty \frac{2}{\sqrt{\pi}} \text{e}^{-\tau^2}\text{d}\tau
\end{equation}
The solution for the discharge vector is
\begin{equation}
Q_x= T\Delta h \frac{2u}{x\sqrt{\pi}} \text{e}^{-u^2} {\hskip 2em} t>t_0
\end{equation}

In [ ]:
# parameters
T = 100 # transmissivity, m^2/d
S = 0.2 # storage coefficient, -
delh = 2 # change in river level, m
t0 = 0 # time of change in river level, d
h0 = 0 # starting head

In [ ]:
# solution
from scipy.special import erfc

# Function for the head
def head(x, t, T, S, delh=1, t0=0):
    u = np.sqrt(S * x ** 2 / (4 * T * (t - t0)))
    return delh * erfc(u)

# Function for the discharge vector
def disvec(x, t, T, S, delh, t0=0):
    u = np.sqrt(S * x ** 2 / (4 * T * (t - t0)))
    return T * delh * 2 * u / (x * np.sqrt(np.pi)) * np.exp(-u ** 2) 

In [ ]:
# plot
x = np.linspace(1e-12, 200, 100)
plt.subplot(121)
for t in [1, 10, 100]:
    h = head(x, t, T, S, delh, t0) + h0
    plt.plot(x, h, label=f'time={t} d')
plt.grid()
plt.xlabel('$x$ (m)')
plt.ylabel('head (m)')
plt.legend()
plt.subplot(122)
for t in [1, 10, 100]:
    Qx = disvec(x, t, T, S, delh, t0)
    plt.plot(x, Qx, label=f'time={t} d')
plt.grid()
plt.xlabel('$x$ (m)')
plt.ylabel('$Q_x$ (m$^2$/d)');

In [ ]:
# basic plot head and Qx vs t
t = np.linspace(1e-12, 100, 100)
plt.subplot(121)
for x in [50, 100, 200]:
    h = head(x, t, T, S, delh, t0)
    plt.plot(t, h, label=f'distance {x} m')
plt.grid()
plt.xlabel('$t$ (d)')
plt.ylabel('head (m)')    
plt.legend()
plt.subplot(122)
for x in [50, 100, 200]:
    Qx = disvec(x, t, T, S, delh, t0)
    plt.plot(t, Qx, label=f'distance {x} m')
plt.grid()
plt.xlabel('$t$ (d)')
plt.ylabel('$Q_x$ (m$^2$/d)');    
plt.legend();

In this flow problem, the head is only a function of the dimensionless parameter $u$, also referred to as the similarity variable (which combines the independent variables). Different combinations of the independent variables $x$, $t$, $T$ and $S$ can give the same head. For example, the head change at $x=100$ m and $t=10$ d is equal to

In [ ]:
print(f'head at x = 100 m, t = 10 d for S = 0.2:')
print(f'h = {head(100, 10, T, 0.2, delh, t0):.6f} m')

If the storage coefficient is not $S=0.2$ but $S=0.002$ and $T = 1$ m$^2$/d instead of $T = 100$ m$^2$/d, the same head is obtained

In [ ]:
print(f'head at x = 100 m, t = 10 d for T = 1 m2/d and S = 0.002:')
print(f'h = {head(100, 10, 1, 0.002, delh, t0):.6f} m')

### Exercise 1.1
At what distance from the canal is the head change at $t=10$ d and $S=0.002$ equal to the head change at $x=10$ m and $t=10$ d when $S=0.2$? Use $T = 100$ m$^2$/d.

### Water balance

The total inflow $Q_{in}$ from the canal into the aquifer over a period $\Delta t$  may be computed through integration of the discharge vector
\begin{equation}
Q_{in} = \int_{t_0}^{t_0+\Delta t} Q_x|_{x=0,t} = 2 \Delta h \sqrt{\frac{ST \Delta t}{\pi}}
\end{equation}
The total inflow must be equal to the total increase in storage. 
The total increase in storage may be computed through numerical integration of the transient head in the aquifer at $t=\Delta t$ and multiplication by the storage coefficient. The integation may be carried out, for example, by using the `quad` method of `scipy.integrate`. The `quad` function returns a number of items of which the first one is the computed integral.

In [ ]:
t = 10 # time period for which water balance is checked
Qin = 2 * delh * np.sqrt(S * T * t / (np.pi)) # from equation
from scipy.integrate import quad
stored = S * quad(head, 1e-12, np.infty, args=(t, T, S, delh, 0))[0]
print(f'total inflow from canal  : {Qin:.6f} m^3')
print(f'total increase in storage: {stored:.6f} m^3')

### Exercise 1.2
Plot the increase in groundwater storage as a function of time between $t = 1$ and $t = 100$ d. Why does the volume keep increasing?

### Effect of a flood wave$^*$
The governing differential equation is linear, which means that superposition of the solutions can be applied to simulate the effect of multiple river level stages, for example the passage of a flood wave. For a single flood wave the boundary condition at the river is
\begin{equation}
h|_{x=0}=h_0 {\hskip 2em} t \le t_0
\end{equation}
\begin{equation}
h|_{x=0}=h_0 + \Delta h {\hskip 2em} t_0\le t \le t_1
\end{equation}
\begin{equation}
h|_{x=0}=h_0  {\hskip 2em} t \ge t_1
\end{equation}

In [ ]:
h0 = 0 # m
delh = 2 # m
t0 = 0
t1 = 4 # days

In [ ]:
plt.figure(figsize=(5, 3))
plt.plot([-1, t0, t0, t1, t1, 3 * t1], [h0, h0, h0 + delh, h0 + delh, h0, h0])
plt.xlabel('time (days)')
plt.ylabel('river stage (m)')
plt.xticks(np.arange(0, 12.1, 4))
plt.grid()

The solution for the head is obtained from superposition as
\begin{equation}
h(x, t) = h_0 + \Delta h \text{erfc}(u_0) {\hskip 2em} t_0\le t \le t_1
\end{equation}
\begin{equation}
h(x, t) = h_0 + \Delta h \text{erfc}(u_0) - \Delta h \text{erfc}(u_1) {\hskip 2em} t \ge t_1
\end{equation}
where
\begin{equation}
u_0 = \sqrt{\frac{Sx^2}{4T(t-t_0)}} {\hskip 2em} u_1 = \sqrt{\frac{Sx^2}{4T(t-t_1)}}
\end{equation}

In [ ]:
# basic plot head and Qx vs x
x = np.linspace(1e-12, 200, 100)
plt.subplot(121)
for t in [t1 + 1e-12, 2 * t1, 3 * t1]:
    h = head(x, t, T, S, delh, t0) - head(x, t, T, S, delh, t1) + h0
    plt.plot(x, h, label=f'time={t:.0f} d')
plt.grid()
plt.xlabel('$x$ (m)')
plt.ylabel('head (m)')
plt.legend()
plt.subplot(122)
t = 2 * t1
Qx = disvec(x, t, T, S, delh, t0) - disvec(x, t, T, S, delh, t1)
plt.plot(x, Qx, 'C1', label=f'time={t} d')
t = 3 * t1
Qx = disvec(x, t, T, S, delh, t0) - disvec(x, t, T, S, delh, t1)
plt.plot(x, Qx, 'C2', label=f'time={t} d')
plt.grid()
plt.xlabel('$x$ (m)')
plt.ylabel('$Q_x$ (m$^2$/d)')
plt.legend();

### Exercise 1.3$^*$
Model the effect of a second flood wave that has $\Delta h = 1$ m and passes between 8 and 10 days. Plot the $h$ and $Q_x$ as a function of $x$ for the same times as in the above figure.

# Solution 2. Transient recharge

## Problem definition
This problem is the transient version of Solution 1 of Session 1. An aquifer is bounded on the left and right sides by two long parallel rivers that fully penetrate the aquifer; for simplicity, the heads in the two rivers are equal to 0. The rivers are a distance $L$ apart and in direct hydraulic contact (i.e., no entry resistance). Initially, the head in the aquifer is equal to the head in the rivers everywhere
\begin{equation}
h\vert_{x=-L/2, t} = h\vert_{x=L/2, t} = h\vert_{x, t=0}
\end{equation}
At time $t=0$, it starts to rain and the groundwater recharge is equal to $N$. 
The transmissivity of the aquifer is approximated as constant and equal to $T$. The storage coefficient is equal to $S$.

<img src="figs/merged_nbs5_67_0.png" width=400>

The solution is somewhat complicated, but the Python script is only a few lines. 
\begin{equation}\label{KvdL}
h = \frac{-N}{2T}\left(x^2 - \tfrac{1}{4}L^2\right) -
\frac{4NL^2}{\pi^3T} \sum_{n=0}^\infty
\frac{(-1)^n}{(2n + 1)^3} \cos\left[\frac{(2n+1)\pi x}{L}\right]
\exp\left[-\frac{(2n+1)^2\pi^2 Tt}{SL^2}\right]
\end{equation}
Note that the first term of the solution is the solution for steady flow (but the origin of the coordinate system is different than for Solution 1 of Session 1):
\begin{equation}
h_\text{steady} = \frac{-N}{2T}\left(x^2 - \tfrac{1}{4}L^2\right)
\end{equation}

In [ ]:
# parameters
L = 1000 # aquifer length, m
S = 0.1 # storage coefficient, -
T = 200 # transmissivity, m^2/d
N = 0.001 # recharge rate, m/d

In [ ]:
# solution
def hsteady(x, T=T, L=L, N=N):
    return -N / (2 * T) * (x ** 2 - L ** 2 / 4)

def head(x, t, T=T, S=S, L=L, N=N, nterms=10):
    h = 0
    for n in range(nterms):
        h += (-1)**n / (2 * n + 1)**3 * \
        np.cos(((2 * n + 1) * np.pi * x) / L) * \
        np.exp(-((2 * n + 1)**2 * np.pi**2 * T * t) / (S * L ** 2))
    h = hsteady(x, T, L, N) - 4 * N * L**2 / (np.pi ** 3 * T) * h
    return h

In [ ]:
# plot
x = np.linspace(-L / 2, L / 2, 100)
for t in [50, 100, 150]:
    plt.plot(x, head(x, t, nterms=10), label=f't={t} d')
plt.plot(x, head(x, 0, nterms=0), 'k', label='steady'); # 0 terms gives steady solution
plt.grid()
plt.xlabel('x (m)')
plt.ylabel('head (m)')
plt.legend();

Next, the head in an observation well at $x=200$ is plotted as a function of time for $t$ varying from 0 to 300 days

In [ ]:
t = np.linspace(0, 300, 100)
h = head(200, t)
plt.plot(t, h)
plt.xlabel('time (d)')
plt.ylabel('head (m)')
plt.grid();

The head in an observation well may be approximated as
\begin{equation}\label{KvdLapprox}
h \approx \frac{-N}{2T}\left(x^2 - \tfrac{1}{4}L^2\right)(1 - \text{e}^{-t/\tau})
\end{equation}
where $\tau$ is the characteristic time of the system, defined as
\begin{equation}\label{aKvdL}
\tau = \frac{SL^2}{\pi^2T}
\end{equation}

### Response time
95% of the final head is reached at $t=3\tau= 3SL^2/(\pi^2 T)$, which is also called the response time (or the memory) of the system. For this case:

In [ ]:
tau = S * L ** 2 / (np.pi ** 2 * T)
t_memory = 3 * tau
print(f'Memory of the system: {t_memory:.0f} d')

This means that if it rains for 1 day, it takes approximaty 152 days before most of the water has left the aquifer and has flown into the river for this example.

### Exercise 2.1
Use both the exact solution and the approximate solution to plot the head in an observation well at $x=200$ vs. time for $t$ varying from 0 to 300 days.

## Block response
The head response due to a recharge event of intensity $N$ from $t_0$ to till $t_1$ is called the block response and is obtained by superposition as
\begin{equation}\label{KvdLapprox2}
h = h_\text{steady}(1 - \text{e}^{-(t-t_0)/\tau}) - 
h_\text{steady}(1 - \text{e}^{-(t-t_1)/\tau}) \qquad t\ge t_1
\end{equation}
which may be simplified to
\begin{equation}\label{KvdLapprox3}
h = h_\text{steady}(\text{e}^{-(t - t_1)/\tau} - \text{e}^{-(t - t_0)/\tau})
\qquad t\ge t_1
\end{equation}

In [ ]:
# parameters
t0 = 0 # start of recharge event, d
t1 = 1 # end of recharge event, d
N = 0.01 # recharge, m/d

In [ ]:
t = np.linspace(t1, 300, 100)
h = hsteady(200, T, L, N) * (np.exp(-(t - t1) / tau) - np.exp(-(t - t0) / tau))
plt.plot(t, h)
plt.xlabel('time (d)')
plt.ylabel('head (m)')
plt.grid()

# Solution 3. Time series analysis of measured head series

The head response due to a series of recharge events is computed through superposition. 
The unit block response, the block response to a recharge event with unit intensity, is written as a function of two parameters: $A$ and $a$ (compare to the block response defined above). 
\begin{equation}\label{KvdLapprox3}
h = A(\text{e}^{-(t - t_1)/a} - \text{e}^{-(t - t_0)/a})
\qquad t\ge t_1
\end{equation}
A Python function is written for the unit block response response. The function takes as input arguments the parameters `A` and `a`, an array of times `t`, and the start of the recharge event `tstart` and the length of the recharge event `delt`. The function returns an array of head responses for all times in the array `t`, where the head is 0 for times before `tstart`.  

In [ ]:
def unitblockresponse(A, a, t, tstart, delt):
    h = np.zeros(len(t))
    h[t >= tstart + delt] = A * (np.exp(-(t[t >= tstart + delt] - (tstart + delt)) / a) - 
                                 np.exp(-(t[t >= tstart + delt] - tstart) / a))
    return h

The head response is plotted for a recharge of $N=0.001$ m/d starting at $t=0$ and ending at $t=30$ days. The head is computed every 30 days up to 180 days. 

In [ ]:
A = 200 # d
a = 40 # d
tstart = 0
delt = 30
t = np.arange(0, 200, 30)
N = 0.001 # m/d
h = N * unitblockresponse(A, a, t, tstart, delt)
plt.plot(t, h, marker='.')
plt.xticks(np.arange(0, 200, delt))
plt.grid()
plt.title('head response to 1 mm/d for 30 days');

As an example, the head response for 5 months of recharge is computed below. First, the head response is plotted for each month separately. 

In [ ]:
t = np.arange(0, 301, 30)
N = 0.001 * np.array([0.4, 1.4, 0.8, 0.2, 1.0]) # 5 months of average daily recharge in mm/d
plt.figure(figsize=(9, 7))
for i in range(5):
    plt.subplot(5, 1, i + 1)
    plt.plot(t, N[i] * unitblockresponse(A, a, t, i * delt, delt))
    plt.xticks(np.arange(0, 301, delt), 11 * [''])
    plt.ylim(0, 0.15)
    if i == 2: plt.ylabel('head (m)')
    plt.grid()
plt.xticks(np.arange(0, 301, delt), np.arange(0, 301, delt))
plt.xlabel('days');

The head response is obtained through superposition by simply adding the five reponses together:

In [ ]:
h = np.zeros(len(t))
for i in range(5):
    h += N[i] * unitblockresponse(A, a, t, i * delt, delt)
plt.plot(t, h)
plt.xticks(np.arange(0, 301, delt), np.arange(0, 301, delt))
plt.xlabel('days')
plt.ylabel('head (m)')
plt.grid();

## Four years of monthly recharge
Consider four years of monthly recharge (a month is defined as 365 days divided by 12 here). The recharge is plotted and the head in the aquifer is computed and plotted at the end of every month for the chosen values of `A` and `a`. 

In [ ]:
recharge = 0.001 * np.array( 
[ 1.2,  1.5,  0.8, -1.2,  0.2, -2.9, -0.6,  2.8,  1.2,  0.9,  3.8,
  1.8,  1.9,  1.8, -0.6, -1.3, -2.5,  0.6,  0.9,  1.8, -0.3,  0.8,
 -0.2,  3.8,  2.8,  0.3, -0.5,  0.3, -1.1,  1.5,  1. , -1.5,  0.1,
  1.5,  1. ,  4. ,  1.3,  1.3,  0.3, -1.1,  0.6, -0.8, -1.2, -1.6,
  1.3,  1.4,  2.4,  2.6]) # monthly recharge, m/d
#
tmax = 4 * 365 # maximum time, d
delt = 365 / 12 # time step, d
t = np.arange(0, tmax + 1, delt) # time, d
#
A = 200 # d
a = 20 # d
h = np.zeros(48 + 1)
for i in range(48):
    h += recharge[i] * unitblockresponse(A, a, t, tstart=i * delt, delt=delt)
#
plt.subplot(211)
plt.step(t, 1000 * np.hstack((recharge, recharge[-1])), where='post')
plt.xticks(np.linspace(0, 4 * 365, 5), 5 * [''])
plt.ylabel('recharge (mm/d)')
plt.grid()
plt.subplot(212)
plt.plot(t, h)
plt.xticks(np.linspace(0, 4 * 365, 5), np.linspace(0, 4, 5))
plt.xlabel('year')
plt.ylabel('head (m)')
plt.grid()

Head observations are available at the end of every month.

In [ ]:
hobs = np.array(
    [ 0.  ,  0.33,  0.6 ,  0.56, -0.01,  0.04, -0.76, -0.61,  0.42,
      0.58,  0.58,  1.36,  1.27,  1.23,  1.19,  0.51, -0.08, -0.74,
     -0.27,  0.1 ,  0.55,  0.24,  0.35,  0.14,  1.11,  1.41,  0.88,
      0.35,  0.27, -0.15,  0.31,  0.46, -0.14, -0.07,  0.37,  0.49,
      1.37,  1.14,  0.99,  0.64,  0.06,  0.18, -0.11, -0.39, -0.66,
      -0.03,  0.38,  0.87,  1.21])

The model is plotted versus the observations. For these values of `A` and `a`, the fit isn't very good. 

In [ ]:
A = 200 # d
a = 20 # d
h = np.zeros(48 + 1)
for i in range(48):
    h += recharge[i] * unitblockresponse(A, a, t, tstart=i * delt, delt=delt)
#
plt.plot(t, hobs, 'k.', label='observed')
plt.plot(t, h, label='model')
plt.xticks(np.linspace(0, 4 * 365, 5), np.linspace(0, 4, 5))
plt.xlabel('year')
plt.ylabel('head (m)')
plt.grid()
plt.legend();

### Exercise 3.1
Find the optimal parameters of `A` and `a` by varying them manually.

## Minimize sum of squared errors
The optimal values of `A` and `a` are obtained by fitting the modeled heads $h_\text{model}$ to the observed heads $h_\text{obs}$. The objective function $f_\text{obj}$ is the sum of squared errors
\begin{equation}
f_\text{obj} = \sum(h_\text{obs} - h_\text{model})^2
\end{equation}
The optimal values of `A` and `a` are obtained by minimizing the  objective function $f_\text{obj}$  using the `fmin` function of `scipy.optimize`. 

In [ ]:
def fobj(p, return_heads=False):
    A, a = p
    h = np.zeros(48 + 1)
    for i in range(48):
        h += recharge[i] * unitblockresponse(A, a, t, tstart=i * delt, delt=delt)
    if return_heads: return h
    return np.sum((h - hobs) ** 2)

In [ ]:
from scipy.optimize import fmin
A, a = fmin(fobj, [200, 20])
print(f'Estimated parameters A: {A:.2f} d, a: {a:.2f} d')
hm = fobj([A, a]) # compute modeled heads

In [ ]:
h = fobj([A, a], return_heads=True) # compute heads with optimal parameters
plt.plot(t, hobs, 'k.', label='observed')
plt.plot(t, h, label='model')
plt.xticks(np.linspace(0, 4 * 365, 5), np.linspace(0, 4, 5))
plt.xlabel('year')
plt.ylabel('head (m)')
plt.grid()
plt.legend();

The above procedure to fit a time series of head observations is referred to as time series analysis. More information on time series analysis can be found in [Collenteur et al., 2019. Pastas: open source software for the analysis of groundwater time series. Groundwater, 57(6), pp.877-885](https://ngwa.onlinelibrary.wiley.com/doi/full/10.1111/gwat.12925). Or check out the [Pastas](https://github.com/pastas/pastas) website. An example analysis of a 10 year time series is shown below, including the model fit and the contribution of rainfall and evaporation to the head variation (from the paper by Collenteur et al., referenced above). 

<img src="figs/fig_pastas.png" width=800>

# Solution 4. Pumping test analysis

### Confined aquifer (Theis)

The Theis solution is a famous formula in groundwater hydrology to compute the drawdown around a well in a confined aquifer. The aquifer is approximated as fully-confined and the transmissivity $T$ and storage coefficient $S$ are approximated as constant. The well extraction rate is $Q$.

<img src="figs/figwell.png"  width=500>

The solution for a well that starts pumping at $t=t_0$ (Theis solution) is
\begin{equation}\label{headtheis}
h = - \frac{Q}{4\pi T}\text{E}_1\left( u\right) {\hskip 2em} t\ge t_0
\end{equation}
where
\begin{equation}
u =  \frac{S r^2}{4T(t-t_0)}
\end{equation}
and E$_1$ is the exponential integral
\begin{equation}
\text{E}_1(u) = \int_u^{\infty} \frac{\exp(-s)}{s}\text{d} s 
\end{equation}

Because the pumped water can only come from storage in the fully confined aquifer, the Theis solution does not reach steady state. It is only suitable to analyze the early stages of a pumping test, before other sources of water (i.e. from an overlying aquifer) start to contribute significantly to the well flow. 

The following example shows how the Theis solution is implemented in Python

In [ ]:
# parameters
T = 200 # transmissivity of aquifer, m^2/d
S = 0.0005 # storage coefficient of aquifer, -
Q = 800 # discharge of well, m^3/d
rw = 0.3 # radius of well, m
t0 = 0 # start of pumping, d

In [ ]:
# solution
from scipy.special import exp1

# Function for equation 15
def theis(r, t, T, S, Q, t0=0):
    u = S * r ** 2 / (4 * T * (t - t0))
    return -Q / (4 * np.pi * T) * exp1(u)

In [ ]:
r = np.linspace(rw, 100, 100)
for i, t in enumerate([1, 2, 10]):
    h = theis(r, t, T, S, Q)
    plt.plot(r, h, 'C' + str(i), label=f't={t} d')
    plt.plot(-r, h, 'C' + str(i))
plt.xlabel('radial distance (m)')
plt.ylabel('head (m)')
plt.legend()
plt.grid()

### Semi-confined aquifer (Hantush)

For flow in a semi-confined aquifer, the pumpted groundwater initially comes from aquifer storage but as the heads drop around the well, water from the overlying aquifer (or surface water) will start to contribute.

<IMG src="figs/merged_nbs6_81_0.png" width=400>

The solution to this problem (for the case that $h^*=0$) is referred to as the Hantush function
\begin{equation}\label{hhantush}
h = -\frac{Q}{4\pi T}\int_u^\infty \frac{1}{\tau}\exp\left(-\tau -\frac{r^2}{4\lambda^2 \tau}\right) \text{d}\tau
\end{equation}
where $\lambda=\sqrt{Tc}$ is the leakage factor, and $u$ is the same as in the Theis solution. For the case that $h^* \ne 0$, it can simply be added to this solution. Much has been written in the literature about how to solve the integral in the Hantush equation but in Python it can simply be calculated using the `quad` function of the `scipy.integrate` package, which we have seen before in Solution 1 of this session.
    
The Hantush and Theis heads are very similar for early times when most of the water pumped by the well comes from storage. The Hantush and Theis heads deviate once a significant amount of water pumped by the well starts to come from leakage through the semi-confining layer. The Theis solution never reaches steady state, but the Hantush solution does. To estimate the time it takes to reach steady state, the Hantush solution can be compared to the solution for steady flow to a well in a semi-confined aquifer, which is
\begin{equation}\label{besselwell}
h = h^* -\frac{Q}{2\pi T} \text{K}_0(r/\lambda)
\end{equation}
in which K$_0$ is the modified Bessel function of the second kind and order 0.

In [ ]:
# parameters
T = 200 # transmissivity of aquifer, m^2/d
S = 0.0005 # storage coefficient of aquifer, -
c = 1000 # resistance of leaky layer, d
Q = 800 # discharge of well, m^3/d
rw = 0.3 # radius of well, m
lab = np.sqrt(c * T) # leakage factor, m
print(f'leakage factor: {lab:.2f} m')

In [ ]:
# solution
from scipy.special import exp1, k0
from scipy.integrate import quad

def integrand(tau, r, T, lab):
    return 1 / tau * np.exp(-tau - r ** 2 / (4 * lab ** 2 * tau))

def hantush(r, t, T, S, c, Q):
    lab = np.sqrt(T * c)
    u = S * r ** 2 / (4 * T * t)
    F = quad(integrand, u, np.inf, args=(r, T, lab))[0]
    return -Q / (4 * np.pi * T) * F

hantushvec = np.vectorize(hantush) # vectorize hantush function

In [ ]:
# basic plot for one value of r and one value of t
plt.subplot(121)
r = np.linspace(rw, 2 * lab, 100)
t = 6 / 24
plt.plot(r, hantushvec(r, t, T, S, c, Q), label='Hantush')
plt.plot(r, -Q / (2 * np.pi * T) * k0(r / lab), 'C0--', label='Steady semi-confined')
plt.plot(r, theis(r, t, T, S, Q), label='Theis') # theis func from above
plt.title('Head after 6 hours')
plt.grid()
plt.xlabel('$x$ (m)')
plt.ylabel('head (m)')
plt.legend()
plt.subplot(122)
t = np.linspace(0.01, 2, 100)
r = lab # evaluate head at r=lambda
plt.plot(t, hantushvec(r, t, T, S, c, Q), label='Hantush')
plt.axhline(-Q / (2 * np.pi * T) * k0(lab / lab), color='C0', ls='--', label='Steady semi-confined')
plt.plot(t, theis(r, t, T, S, Q), '--', label='Theis')
plt.title(f'Head at {lab:0.1f} m')
plt.grid()
plt.xlabel('$t$ (d)')
plt.ylabel('head (m)');

### Pumping test data

In [ ]:
# Data at Oude Korendijk taken from Kruseman and de Ridder (1990)
robs = 30 # distance to of observation well, m
Q = 788 # discharge of well, m^3/d
time = np.array(
    [0.1 ,  0.25,  0.5 ,  0.7 ,  1.  ,  1.4 ,  1.9 ,  2.33,  2.8 ,
     3.36,  4.  ,  5.35,  6.8 ,  8.3 ,  8.7 , 10.  , 13.1 , 18.  ,
     27. ,  33. ,  41. ,  48. ,  59. ,  80. , 95.  , 139. , 181. , 
     245.,  300.,  360.,  480.,  600.,  728., 830.]) # in minutes
drawdown = np.array(
    [0.04 , 0.08 , 0.13 , 0.18 , 0.23 , 0.28 , 0.33 , 0.36 , 0.39 ,
     0.42 , 0.45 , 0.5  , 0.54 , 0.57 , 0.58 , 0.6  , 0.64 , 0.68 ,
     0.742, 0.753, 0.779, 0.793, 0.819, 0.855, 0.873, 0.915, 0.935,
     0.966, 0.99 , 1.007, 1.05 , 1.053, 1.072, 1.088]) # in meters
tobs = time / 24 / 60 # convert observation time to days
hobs = -drawdown # convert drawdown to heads

In [ ]:
plt.subplot(121)
plt.plot(tobs, hobs, 'C1.')
plt.xlabel('time (d)')
plt.ylabel('head (m)')
plt.title(f'measured head at r={robs} m')
plt.grid()

Find optimal values for transmissivity $T$ and storage coefficient $S$ using least squares.

In [ ]:
# Define objective function to minimize
def fobj(p, ho, to, ro, Q):
    T, S = np.exp(p)
    hm = theis(ro, to, T, S, Q)
    return np.sum((ho - hm) ** 2)

In [ ]:
# least squares solution
from scipy.optimize import fmin
logp = fmin(fobj, np.log(np.array([100, 1e-4])), args=(hobs, tobs, robs, Q), disp=True)
T, S = np.exp(logp)
print(f'Estimated parameters T: {T:.1f} m^2/d, S: {S:.2e}')

In [ ]:
# plot of best fit
hm = theis(robs, tobs, T, S, Q)
plt.subplot(121)
plt.plot(tobs, hobs, 'C1.')
plt.plot(tobs, hm, 'C0')
plt.xlabel('time (d)')
plt.ylabel('head (m)')
plt.grid()
plt.subplot(122)
plt.semilogx(tobs, hobs, 'C1.')
plt.semilogx(tobs, hm, 'C0')
plt.xlabel('time (d)')
plt.ylabel('head (m)')
plt.grid();

## Exercise 4.1

Fit the pump test data using the Hantush formula. You can do that by trial and error (varying the values of the parameters $T$, $S$ and $c$ by hand), or by formally minimizing the sum of the squared errors (note that you may see some warnings and that it could take a while before the calculation is finished).